代码看着长，但是其实并不是很长，分为三大块主要内容：  
- 前向传播的过程  
- 反向传播更新参数的过程  
- 预测  

备注：面试题一般只会考单隐层的神经网络结构，而单层的神经网络结构有无法对复杂的非线性函数进行很好的拟合（主要是因为单隐层的神经网络没办法非线性，主要是取决于激活函数），所以要考的话也只能是单层的线性函数的拟合，这个是可以解决的（不在后面添加激活函数即可）；重点还是**反向传播求导更新参数**的过程。

In [1]:
import numpy as np

##### 定义sigmoide函数

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [3]:
sigmoid(np.random.randint(1,3,10))

array([0.88079708, 0.88079708, 0.88079708, 0.73105858, 0.73105858,
       0.73105858, 0.88079708, 0.88079708, 0.73105858, 0.88079708])

##### 初始化参数
将参数初始化为0

In [4]:
def initialize_with_zeros(dim):
    """
    参数：
    dim：向量的维度，或特征的个数。
    返回值
    w：初始化的向量，维度(dim, 1)
    b：初始化的截距。
    """
    w = np.zeros(shape=(dim,1))
    b = 0
    assert w.shape == (dim,1)
    assert isinstance(b, int) or isinstance(b,float)
    return w,b

In [5]:
dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))

w = [[0.]
 [0.]]
b = 0


##### 前向传播
前序传播和反向传播函数是用来学习参数的函数。其过程如下：

- 传入$X$
- 计算 $A = \sigma(w^T X + b) = (a^{(0)}, a^{(1)}, ..., a^{(m-1)}, a^{(m)})$
- 计算损失函数 $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$
- 计算参数的相对梯度（一阶导数）。

In [6]:
# 传播函数
def propagate(w, b, X, Y):
    """
    参数：
    w：数组，权重。
    b：浮点型，偏差值。
    X：特征数据
    Y：数据标签。
    Return:
    cost：损失值
    dw：关于参数w的梯度，维度与w相同。
    db：关于参数b的梯度，维度与b相同。
    """
    m = X.shape[1]
    # 前向传播
    A = sigmoid(np.dot(w.T,X)+b)    
    cost = -1./m * np.sum(np.multiply(np.log(A), Y) + np.multiply(np.log(1-A),1-Y))# compute cost
    
    dw = None
    db = None
    # 反向传播
    dZ = A-Y
    dw = 1./m * np.dot(dZ, X.T)
    dw = dw.T
    db = 1./m * np.sum(dZ, keepdims=True, axis=1)

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    grads = {"dw": dw,
             "db": db}
    return grads, cost

In [7]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[0.99845601]
 [2.39507239]]
db = [[0.00145558]]
cost = 5.801545319394553


##### 优化参数

目标是通过最小化损失函数$J$来学习$w$和$b$。 对参数$\theta$，更新规则$ \theta = \theta - \alpha \text{ } d\theta$, 其中$\alpha$是学习率。

In [ ]:
# 优化函数
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    参数：
    w：数组，权重。
    b：浮点型，偏差值。
    X：数组，特征数据
    Y：数组， 数据标签。
    num_iterations：整型，迭代次数
    learning_rate：浮点型，学习率
    print_cost：布尔型，是否打印损失值。
    Returns:
    params：字典，参数w和b
    grads：字典，参数w和b的梯度
    costs：列表，记录迭代过程中的损失值。
    """
    costs = []
    for i in range(num_iterations):
        # 损失和梯度计算
        grads, cost = propagate(w,b,X,Y)
        # 相对梯度
        dw = grads['dw']
        db = grads['db']
        # 更新参数
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # 记录损失值
        if i % 100 == 0:
            costs.append(cost)
        # 打印损失值
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    params = {"w": w,
              "b": b}
    grads = {"dw": dw,
             "db": db}
    return params, grads, costs

##### 预测
通过公式$\hat{Y} = A = \sigma(w^T X + b)$进行预测，设定0.5为阈值。


In [12]:
# GRADED FUNCTION: predict
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    Arguments:
    w：数组，权重
    b：标量，截距
    X：数组，带预测数据
    Returns:
    Y_prediction：数组，预测结果（非概率结果，是分类结果。大于或等于0.5为1，小于0.5为0）
    '''
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # 计算预测值，并得出分类
    A = sigmoid(np.dot(w.T,X)+b)
    for i in range(X.shape[0]):
        Y_prediction[0,i] = 0 if A[0][i]<0.5 else 1
    assert(Y_prediction.shape == (1, m))
    print(A)
    return Y_prediction

In [11]:
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))

[[0.52241976 0.50960677 0.34597965]]
predictions = [[1. 1. 0.]]
